In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from pytorch_pretrained_bert.modeling import BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]
EMBEDDING_DIM = 768 #1024
# EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()


Found 3427 unique tokens


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5744
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4596:], batch_size=32)
    np.save('5744npy-BERT/text_5744-BERT-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy-BERT/text-BERT_5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 159s 35ms/step - loss: 2.7636 - v_loss: 0.8231 - a_loss: 0.9642 - d_loss: 0.9740 - v_ccc: 0.1758 - a_ccc: 0.0352 - d_ccc: 0.0254 - val_loss: 2.6445 - val_v_loss: 0.8539 - val_a_loss: 0.8993 - val_d_loss: 0.8908 - val_v_ccc: 0.1459 - val_a_ccc: 0.1008 - val_d_ccc: 0.1088
Epoch 2/50
4595/4595 [==============================] - 158s 34ms/step - loss: 2.1136 - v_loss: 0.5801 - a_loss: 0.7384 - d_loss: 0.7959 - v_ccc: 0.4200 - a_ccc: 0.2621 - d_ccc: 0.2043 - val_loss: 2.7270 - val_v_loss: 0.8642 - val_a_loss: 0.9095 - val_d_loss: 0.9534 - val_v_ccc: 0.1358 - val_a_ccc: 0.0909 - val_d_ccc: 0.0464
Epoch 3/50
4595/4595 [==============================] - 160s 35ms/step - loss: 1.8025 - v_loss: 0.5033 - a_loss: 0.6213 - d_loss: 0.6780 - v_ccc: 0.4968 - a_ccc: 0.3788 - d_ccc: 0.3220 - val_loss: 2.7057 - val_v_loss

Epoch 13/50
4595/4595 [==============================] - 162s 35ms/step - loss: 0.8723 - v_loss: 0.2446 - a_loss: 0.3092 - d_loss: 0.3175 - v_ccc: 0.7551 - a_ccc: 0.6903 - d_ccc: 0.6822 - val_loss: 2.7325 - val_v_loss: 0.8800 - val_a_loss: 0.9047 - val_d_loss: 0.9476 - val_v_ccc: 0.1196 - val_a_ccc: 0.0955 - val_d_ccc: 0.0524
Epoch 14/50
1422/1422 [==============================] - 9s 7ms/step
[0.19689911603927612, 0.2133343368768692, 0.15392789244651794]
1
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 164s 36ms/step - loss: 2.7201 - v_loss: 0.7940 - a_loss: 0.9616 - d_loss: 0.9638 - v_ccc: 0.2058 - a_ccc: 0.0380 - d_ccc: 0.0361 - val_loss: 2.6641 - val_v_loss: 0.8397 - val_a_loss: 0.9237 - val_d_loss: 0.8994 - val_v_ccc: 0.1599 - val_a_ccc: 0.0758 - val_d_ccc: 0.1002
Epoch 2/50
4595/4595 [==============================] - 162s 35ms/step - loss: 2.0762 - v_loss: 0.5636 - a_loss: 0.7404 - d_loss: 0.7715 - v_ccc: 0.4359 - a_ccc: 0

Epoch 12/50
4595/4595 [==============================] - 157s 34ms/step - loss: 0.9401 - v_loss: 0.2582 - a_loss: 0.3327 - d_loss: 0.3496 - v_ccc: 0.7421 - a_ccc: 0.6672 - d_ccc: 0.6506 - val_loss: 2.7630 - val_v_loss: 0.8743 - val_a_loss: 0.9243 - val_d_loss: 0.9637 - val_v_ccc: 0.1253 - val_a_ccc: 0.0757 - val_d_ccc: 0.0360
Epoch 13/50
1422/1422 [==============================] - 9s 6ms/step
[0.21058015525341034, 0.15422360599040985, 0.11638814210891724]
3
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 160s 35ms/step - loss: 2.7580 - v_loss: 0.8111 - a_loss: 0.9705 - d_loss: 0.9751 - v_ccc: 0.1878 - a_ccc: 0.0293 - d_ccc: 0.0249 - val_loss: 2.8314 - val_v_loss: 0.9316 - val_a_loss: 0.9245 - val_d_loss: 0.9746 - val_v_ccc: 0.0684 - val_a_ccc: 0.0751 - val_d_ccc: 0.0250
Epoch 2/50
4595/4595 [==============================] - 160s 35ms/step - loss: 2.1417 - v_loss: 0.5785 - a_loss: 0.7610 - d_loss: 0.8010 - v_ccc: 0.4212 - a_ccc: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 160s 35ms/step - loss: 1.8034 - v_loss: 0.4932 - a_loss: 0.6331 - d_loss: 0.6768 - v_ccc: 0.5062 - a_ccc: 0.3673 - d_ccc: 0.3231 - val_loss: 2.6631 - val_v_loss: 0.8873 - val_a_loss: 0.8842 - val_d_loss: 0.8909 - val_v_ccc: 0.1124 - val_a_ccc: 0.1159 - val_d_ccc: 0.1086
Epoch 4/50
4595/4595 [==============================] - 160s 35ms/step - loss: 1.6170 - v_loss: 0.4387 - a_loss: 0.5668 - d_loss: 0.6115 - v_ccc: 0.5608 - a_ccc: 0.4337 - d_ccc: 0.3885 - val_loss: 2.6874 - val_v_loss: 0.8578 - val_a_loss: 0.9000 - val_d_loss: 0.9290 - val_v_ccc: 0.1417 - val_a_ccc: 0.1002 - val_d_ccc: 0.0707
Epoch 5/50
4595/4595 [==============================] - 159s 35ms/step - loss: 1.4307 - v_loss: 0.3885 - a_loss: 0.5030 - d_loss: 0.5379 - v_ccc: 0.6110 - a_ccc: 0.4967 - d_ccc: 0.4616 - val_loss: 2.7897 - val_v_loss: 0.8414 - val_a_loss: 0.9408 - val_d_loss: 1.0072 - val_v_ccc: 0.1581 - val_a_ccc: 0.0593 - val_d_ccc: -0.0072
Epoch 6/50
4595/4595 [=======

...

4595/4595 [==============================] - 159s 35ms/step - loss: 1.4034 - v_loss: 0.3914 - a_loss: 0.4908 - d_loss: 0.5195 - v_ccc: 0.6084 - a_ccc: 0.5085 - d_ccc: 0.4797 - val_loss: 2.7726 - val_v_loss: 0.8585 - val_a_loss: 0.9322 - val_d_loss: 0.9820 - val_v_ccc: 0.1411 - val_a_ccc: 0.0681 - val_d_ccc: 0.0182
Epoch 6/50
4595/4595 [==============================] - 159s 35ms/step - loss: 1.2928 - v_loss: 0.3598 - a_loss: 0.4521 - d_loss: 0.4810 - v_ccc: 0.6400 - a_ccc: 0.5479 - d_ccc: 0.5194 - val_loss: 2.7373 - val_v_loss: 0.8587 - val_a_loss: 0.9038 - val_d_loss: 0.9745 - val_v_ccc: 0.1407 - val_a_ccc: 0.0964 - val_d_ccc: 0.0256
Epoch 7/50
4595/4595 [==============================] - 161s 35ms/step - loss: 1.2005 - v_loss: 0.3263 - a_loss: 0.4239 - d_loss: 0.4496 - v_ccc: 0.6738 - a_ccc: 0.5757 - d_ccc: 0.5501 - val_loss: 2.6819 - val_v_loss: 0.8332 - val_a_loss: 0.9072 - val_d_loss: 0.9410 - val_v_ccc: 0.1663 - val_a_ccc: 0.0930 - val_d_ccc: 0.0588
Epoch 8/50
4595/4595 [========

4595/4595 [==============================] - 160s 35ms/step - loss: 1.1910 - v_loss: 0.3328 - a_loss: 0.4213 - d_loss: 0.4381 - v_ccc: 0.6675 - a_ccc: 0.5789 - d_ccc: 0.5625 - val_loss: 2.7603 - val_v_loss: 0.8535 - val_a_loss: 0.9356 - val_d_loss: 0.9703 - val_v_ccc: 0.1461 - val_a_ccc: 0.0642 - val_d_ccc: 0.0294
Epoch 8/50
4595/4595 [==============================] - 158s 34ms/step - loss: 1.1338 - v_loss: 0.3129 - a_loss: 0.4042 - d_loss: 0.4176 - v_ccc: 0.6870 - a_ccc: 0.5962 - d_ccc: 0.5830 - val_loss: 2.7699 - val_v_loss: 0.8567 - val_a_loss: 0.9335 - val_d_loss: 0.9790 - val_v_ccc: 0.1428 - val_a_ccc: 0.0666 - val_d_ccc: 0.0207
Epoch 9/50
4595/4595 [==============================] - 159s 35ms/step - loss: 1.0736 - v_loss: 0.2904 - a_loss: 0.3844 - d_loss: 0.3994 - v_ccc: 0.7095 - a_ccc: 0.6158 - d_ccc: 0.6011 - val_loss: 2.7085 - val_v_loss: 0.8737 - val_a_loss: 0.9020 - val_d_loss: 0.9316 - val_v_ccc: 0.1257 - val_a_ccc: 0.0977 - val_d_ccc: 0.0680
Epoch 10/50
4595/4595 [=======

4595/4595 [==============================] - 158s 34ms/step - loss: 1.1157 - v_loss: 0.3086 - a_loss: 0.3905 - d_loss: 0.4175 - v_ccc: 0.6916 - a_ccc: 0.6097 - d_ccc: 0.5830 - val_loss: 2.7334 - val_v_loss: 0.8604 - val_a_loss: 0.9076 - val_d_loss: 0.9646 - val_v_ccc: 0.1389 - val_a_ccc: 0.0925 - val_d_ccc: 0.0352
Epoch 9/50
4595/4595 [==============================] - 160s 35ms/step - loss: 1.0658 - v_loss: 0.2965 - a_loss: 0.3733 - d_loss: 0.3952 - v_ccc: 0.7033 - a_ccc: 0.6264 - d_ccc: 0.6044 - val_loss: 2.7228 - val_v_loss: 0.8654 - val_a_loss: 0.9042 - val_d_loss: 0.9525 - val_v_ccc: 0.1338 - val_a_ccc: 0.0959 - val_d_ccc: 0.0475
Epoch 10/50
4595/4595 [==============================] - 157s 34ms/step - loss: 1.0082 - v_loss: 0.2814 - a_loss: 0.3527 - d_loss: 0.3751 - v_ccc: 0.7196 - a_ccc: 0.6475 - d_ccc: 0.6248 - val_loss: 2.7589 - val_v_loss: 0.8620 - val_a_loss: 0.9344 - val_d_loss: 0.9623 - val_v_ccc: 0.1372 - val_a_ccc: 0.0659 - val_d_ccc: 0.0380
Epoch 11/50
1422/1422 [======

4595/4595 [==============================] - 160s 35ms/step - loss: 1.0726 - v_loss: 0.2960 - a_loss: 0.3781 - d_loss: 0.3989 - v_ccc: 0.7043 - a_ccc: 0.6219 - d_ccc: 0.6012 - val_loss: 2.7795 - val_v_loss: 0.8746 - val_a_loss: 0.9150 - val_d_loss: 0.9902 - val_v_ccc: 0.1250 - val_a_ccc: 0.0853 - val_d_ccc: 0.0101
Epoch 10/50
4595/4595 [==============================] - 159s 35ms/step - loss: 1.0265 - v_loss: 0.2811 - a_loss: 0.3638 - d_loss: 0.3822 - v_ccc: 0.7188 - a_ccc: 0.6365 - d_ccc: 0.6181 - val_loss: 2.7179 - val_v_loss: 0.8590 - val_a_loss: 0.8982 - val_d_loss: 0.9605 - val_v_ccc: 0.1407 - val_a_ccc: 0.1021 - val_d_ccc: 0.0392
Epoch 11/50
4595/4595 [==============================] - 160s 35ms/step - loss: 0.9765 - v_loss: 0.2669 - a_loss: 0.3474 - d_loss: 0.3623 - v_ccc: 0.7329 - a_ccc: 0.6527 - d_ccc: 0.6378 - val_loss: 2.7659 - val_v_loss: 0.8554 - val_a_loss: 0.9087 - val_d_loss: 1.0016 - val_v_ccc: 0.1442 - val_a_ccc: 0.0916 - val_d_ccc: -0.0017
Epoch 12/50
1422/1422 [====

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)